In [1]:
useMock=True

In [2]:
from cffi import FFI
ffi = FFI()
with open('extracted_headers/safe_c_ffi_data.h','r') as f:
    myHeaders=f.read()
ffi.cdef(myHeaders)
with open('extracted_headers/safe_c_ffi_funcs.h','r') as f:
    funs=f.read()
ffi.cdef(funs)
lib=ffi.dlopen('../compiled_binaries/libsafe_app' + useMock*'_mock' + '.so')

---

maybe it makes most sense to really define the data types by hand and just check them for changes with updates on the api - not entirely sure

- rid

---

```c
typedef unsigned long int uintptr_t;

typedef struct {
 char* id;
 char* scope;
 char* name;
 char* vendor;
} AppExchangeInfo;
```

In [3]:
class AppExchangeInfo:
    def __init__(self,id=b'noId',scope=b'noScope',name=b'noName',vendor=b'nobody'):
        self.id = ffi.new('char[]',id)
        self.scope = ffi.new('char[]',scope)
        self.name = ffi.new('char[]',name)
        self.vendor = ffi.new('char[]',vendor)

        self.entity = ffi.new('AppExchangeInfo *',[self.id,self.scope,self.name,self.vendor])

In [4]:
def AppExchangeInfo(id=b'noId',scope=b'noScope',name=b'noName',vendor=b'nobody'):
    id = ffi.new('char[]',id)
    scope = ffi.new('char[]',scope)
    name = ffi.new('char[]',name)
    vendor = ffi.new('char[]',vendor)

    myStruct = ffi.new('AppExchangeInfo *',[id,scope,name,vendor])
    
    return myStruct, [id, scope, name, vendor]

newExChangeInfo,infopayload = AppExchangeInfo(b'0.0.2',name=b'pyTest',vendor=b'rid')

In [5]:
newExChangeInfo,infopayload = AppExchangeInfo(id=b'net.maidsafe.tutorials.nodejs',scope=b'0.0.1',
                                              name=b'PyTest never ever',vendor=b'riddim from over there')

In [6]:
ffi.string(newExChangeInfo.vendor)

b'riddim from over there'

In [7]:
ffi.string(newExChangeInfo.name)

b'PyTest never ever'

```c
typedef struct {
 _Bool read;
 _Bool insert;
 _Bool update;
 _Bool delete;
 _Bool manage_permissions;
} PermissionSet;
```

In [8]:
def PermissionSet(read=True,insert=True,update=True,delete=True,manage_permissions=True):
    return ffi.new('PermissionSet *',[read,insert,update,delete,manage_permissions])

In [9]:
newPermissionSet = PermissionSet()

```c
typedef struct {
 char* cont_name;
 PermissionSet access;
} ContainerPermissions;
```

In [10]:
def ContainerPermissions(name=b'noName',access=None):
    containerName = ffi.new('char[]',name)
    if not access:
        access = PermissionSet()
    container = ffi.new('ContainerPermissions *',[containerName,access[0]])
    
    return container, [containerName,access]

In [11]:
newPermissions,permission_addon=ContainerPermissions(b'containerPy')

In [12]:
newPermissions.access.insert

True

```c
typedef struct {
 AppExchangeInfo app;
 _Bool app_container;
 ContainerPermissions* containers;
 uintptr_t containers_len;
 uintptr_t containers_cap;
} AuthReq;
```

In [13]:
def AuthReq(permissions,containers_len,containers_cap,id=b'noId',scope=b'pythonscript',
            name=b'noName',vendor=b'nobody',app_container=True):
    
    newExChangeInfo,infopayload = AppExchangeInfo(id,scope,name,vendor)
    
    authReq = ffi.new('AuthReq *',[newExChangeInfo[0],app_container,permissions,containers_len,containers_cap])
    
    return authReq, [newExChangeInfo,infopayload]

In [14]:
myAuth,addData=AuthReq(newPermissions,0,0,id=b'ridsTest',scope=b'pythonscript'
                       ,name=b'PyTest',vendor=b'rid never work',app_container=True)

In [15]:
@ffi.callback("void(void *, FfiResult *)")
def feedback(userData, result):
    print(userData)
    print(result.error_code)

In [16]:
lib.app_set_additional_search_path(ffi.new('char[]',b'../compiled_binaries/'),ffi.NULL,feedback)

<cdata 'void *' NULL>
0


@ffi.callback("void(void *, FfiResult *, unsigned int, char *)")
def o_cb(userData, result, noClueWhatNumber, resultString):
    print(userData)
    print(result)
    print(noClueWhatNumber)
    print(resultString)

lib.encode_auth_req(myAuth,ffi.NULL,o_cb)

myAuth.containers.access.insert=True

myAuth.containers.access.delete=True

myAuth.containers.access.read=True

In [17]:
ffi.string(myAuth.app.name)

b'PyTest'

In [18]:
ffi.string(myAuth.app.id)

b'ridsTest'

In [19]:
encodedAuth=None

In [20]:
@ffi.callback("void(void *, FfiResult *, unsigned int, char *)")
def o_cb(userData, result, reqId, encodedReq):
    global encodedAuth
    print(userData)
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(reqId)
    print(ffi.string(encodedReq))
    encodedAuth = ffi.string(encodedReq)

In [21]:
lib.encode_auth_req(myAuth,ffi.NULL,o_cb)

<cdata 'void *' NULL>
0
3487334486
b'AAAAAFaA3M8AAAAACAAAAAAAAAByaWRzVGVzdAEMAAAAAAAAAHB5dGhvbnNjcmlwdAYAAAAAAAAAUHlUZXN0DgAAAAAAAAByaWQgbmV2ZXIgd29yawEAAAAAAAAAAA'


In [22]:
b'safe-auth://' + encodedAuth

b'safe-auth://AAAAAFaA3M8AAAAACAAAAAAAAAByaWRzVGVzdAEMAAAAAAAAAHB5dGhvbnNjcmlwdAYAAAAAAAAAUHlUZXN0DgAAAAAAAAByaWQgbmV2ZXIgd29yawEAAAAAAAAAAA'

In [23]:
import base64

In [24]:
schemeName=b'safe-'+base64.encodebytes(ffi.string(myAuth.app.id))[:-2]

In [25]:

sysUri = FFI()
with open('extracted_headers/safe_c_ffi_data.h','r') as f:
    myHeaders=f.read()
sysUri.cdef(myHeaders)
with open('extracted_headers/system_uri.h','r') as f:
    funs=f.read()
sysUri.cdef(funs)
lib2=sysUri.dlopen('../compiled_binaries/libsystem_uri.so')

In [26]:
@sysUri.callback("void(void *, FfiResult *)")
def installerFeedback(userData, result):
    print(userData)
    print(result.error_code)

```c
void install(char* bundle, 
             char* vendor, 
             char* name, 
             char* exec_args, 
             uint_least64_t exec_args_len, 
             char* icon, 
             char* schemes, 
             void* user_data, 
             void (*o_cb)(void* user_data, FfiResult* result));
```

In [27]:
print(ffi.string(myAuth.app.name))
print(ffi.string(myAuth.app.vendor))
print(ffi.string(myAuth.app.scope))

b'PyTest'
b'rid never work'
b'pythonscript'


In [28]:
import os
print(os.getcwd())

/home/riddim/maidsafe/pySafe/pySafe


In [29]:
pathToHandler=(os.getcwd()+'uriHandler.py').encode()

In [30]:
pathToHandler

b'/home/riddim/maidsafe/pySafe/pySafeuriHandler.py'

In [31]:
bundle=sysUri.new('char[]',ffi.string(myAuth.app.id))
vendor=sysUri.new('char[]',ffi.string(myAuth.app.vendor))
name=sysUri.new('char[]',ffi.string(myAuth.app.name))
exec_args=sysUri.new('char[]',b'python')
exec_args2=sysUri.new('char[]',pathToHandler)
exec_args_len=2
icon=sysUri.new('char[]',b'')
schemes=sysUri.new('char[]',schemeName)
user_data=sysUri.NULL

In [32]:
exec_args_1 = sysUri.new('char*[]',[exec_args,exec_args2])

In [33]:
lib2.install(bundle,vendor,name,exec_args_1 ,exec_args_len,icon,schemes,user_data,installerFeedback)

<cdata 'void *' NULL>
0


```c 
void open_uri(char* uri, 
              void* user_data, 
              void (*o_cb)(void* user_data, FfiResult* result));
```

In [34]:
@sysUri.callback("void(void *, FfiResult *)")
def openerFeedback(userData, result):
    global res
    res = result
    print(userData)
    print(result.error_code)

In [35]:
ffi.string(myAuth.app.name)

b'PyTest'

In [36]:
ffi.string(myAuth.app.vendor)

b'rid never work'

In [37]:
ffi.string(myAuth.app.name)

b'PyTest'

In [38]:
uri=sysUri.new('char[]',b'safe-auth://' + encodedAuth)
user_data=sysUri.NULL

In [39]:
lib2.open_uri(uri,user_data,openerFeedback)

<cdata 'void *' NULL>
0


```c 
void decode_ipc_msg(char* msg, 
                    void* user_data, 
                    void (*o_auth)(void* user_data, uint32_t req_id, AuthGranted* auth_granted), 
                    void (*o_unregistered)(void* user_data, uint32_t req_id, uint8_t* serialised_cfg, 
                                            uintptr_t serialised_cfg_len), 
                    void (*o_containers)(void* user_data, uint32_t req_id), 
                    void (*o_share_mdata)(void* user_data, uint32_t req_id), 
                    void (*o_revoked)(void* user_data), 
                    void (*o_err)(void* user_data, FfiResult* result, uint32_t req_id));
```

In [40]:
import time

In [41]:
grantedAuth=None
innerVals=None
anotherVal=None
@ffi.callback("void(void *, uint32_t, AuthGranted *)")
def o_auth_decode_ipc_msg(userData, req_id, auth_granted):
    global grantedAuth
    global innerVals
    global anotherVal
    grantedAuth = auth_granted
    print('o_auth')
    print(userData)
    print(req_id)

In [42]:
@ffi.callback("void(void *, uint32_t, uint8_t *, uintptr_t)")
def o_unregistered_decode_ipc_msg(userData, req_id, serialised_cfg, serialised_cfg_len):
    print('o_unregistered')
    print(req_id)

In [43]:
@ffi.callback("void(void *, uint32_t)")
def o_containers_decode_ipc_msg(userData, req_id):
    print('o_containers')
    print(req_id)

In [44]:
@ffi.callback("void(void *, uint32_t)")
def o_share_mdata_decode_ipc_msg(userData, req_id):
    print('o_share')
    print(req_id)

In [45]:
@ffi.callback("void(void *)")
def o_revoked_decode_ipc_msg(userData):
    print('o_revoked')
    print(req_id)

In [46]:
@ffi.callback("void(void *, FfiResult *, uint32_t)")
def o_error_decode_ipc_msg(userData, result, req_id):
    global res
    res = result
    print('o_err')
    print(userData)
    print(req_id)
    print(result.error_code)
    print(ffi.string(result.description))

In [47]:
with open('answer','r') as f:
    answer=f.readlines()

In [48]:
import re
data=re.compile("'safe-.+?:(.+?)'")

In [49]:
my=data.findall(answer[-1])

In [50]:
msg = ffi.new('char[]',my[-1].encode())

In [51]:
my[-1]

'AQAAAIoBIfgAAAAAAAAAACAAAAAAAAAAFPQGLSdsVchlMKaX25kVQ4-n_UYx6JCq_YEzrG2U6D0gAAAAAAAAAFJbpubdtxmnWpp36TUd4PJdCcEatr_6aMckmKE6KkGYIAAAAAAAAABSuFz4NeMk-IDyYZneNZ5PRWo5XswEp51XRkEEIFE2mUAAAAAAAAAA_1HfcHLZcvmghE8-gzZe1VNGtNZx0b7NYhk77KQ7iBFSuFz4NeMk-IDyYZneNZ5PRWo5XswEp51XRkEEIFE2mSAAAAAAAAAAzfFpdP7KOBUPzTXWazTEorsdQOLs-gCjDmydSOb5ljAgAAAAAAAAAHl8wGPQAW1hnrJ13ObJLzMGdo4l3_KoaiIZmIRSdj2WAAAAAAAAAAAAAAAAAAAAAH-RX6Jgcj0mGZo1ZhGdiBPw3LtXTwaog2MV-IlilODcmDoAAAAAAAAYAAAAAAAAAIZpVRK736kjFA-mE1vEhozs_nEBesosCAAAAAAAAAAA'

In [52]:
lib.decode_ipc_msg(msg,ffi.NULL,o_auth_decode_ipc_msg,o_unregistered_decode_ipc_msg
                   ,o_containers_decode_ipc_msg,o_share_mdata_decode_ipc_msg,
                   o_revoked_decode_ipc_msg,o_error_decode_ipc_msg)

o_auth
<cdata 'void *' NULL>
4162912650


----

now the app

In [53]:
myApp = None

In [54]:
@ffi.callback("void(void*)")
def disconnect_app_registered(user_data):
    print('disconnected - callback from app_registered')

In [55]:
@ffi.callback("void(void* , FfiResult* , App* app)")
def result_app_registered(user_data, result, app):
    print('results')
    global myApp
    myApp = app
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))

In [56]:
myAuth.app.id

<cdata 'char *' 0x562273b148b0>

In [57]:
grantedAuth

<cdata 'AuthGranted *' 0x7ffda43be560>

In [58]:
lib.app_registered(myAuth.app.id,grantedAuth,ffi.NULL,disconnect_app_registered,result_app_registered)

results
-1
b'Serialisation error'


---

## looking into the granted authentication

In [72]:
def printArray(array):
    arr=[]
    for element in array:
        arr.append(element)
    print(arr)

In [59]:
grantedAuth.app_keys.owner_key

<cdata 'unsigned char[32]' 0x7ffea2566a90>

In [60]:
len(grantedAuth.app_keys.owner_key)

32

In [73]:
printArray(grantedAuth.app_keys.owner_key)

[0, 0, 0, 0, 0, 0, 0, 0, 176, 132, 122, 173, 182, 85, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 174, 134, 144, 171, 182, 85, 0, 0]


In [62]:
len(grantedAuth.app_keys.enc_key)

32

In [63]:
len(grantedAuth.app_keys.sign_pk)

32

In [64]:
len(grantedAuth.app_keys.sign_sk)

64

In [65]:
len(grantedAuth.app_keys.enc_pk)

32

In [66]:
len(grantedAuth.app_keys.enc_sk)

32